In [ ]:
import pymysql.cursors
cnx = pymysql.connect(host='localhost',
                             user='root',
                             password='gzy1589745',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [ ]:
class CSVTable:
    def __init__(self, table_name, primary_key):
        self.table_name = table_name 
        self.primary_key = primary_key
        
    def DBRunQuery(self,q):
        cursor=cnx.cursor()
        print ("Query = ", q)
        cursor.execute(q);
        r = cursor.fetchall()
        return r

    def templateToWhereClause(self,t):
        s = ""
        for (k,v) in t.items():
            if s != "":
                s += " AND "
            s += k + "='" + v + "'"

        if s != "":
            s = "WHERE " + s;
        return s
    def fieldsToColumName(self,fields):
        if fields == None:
            field = "* "
        else:
            field = ""
            for f in fields:
                field += f + ","
        return field[:-1]
    
    def templateToKeysValues(self,t):
        s1 = "("
        s2 = "("
        for (k,v) in t.items():
            s1 += k + ","
            s2 += "'" + v + "',"
        s1 = s1[:-1] + ")"
        s2 = s2[:-1] + ")"
        s = [s1,s2]
        return s
    
    def primary_key_error(self):
        q = "SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='" + self.table_name + "'"
        columns = [dic.get('COLUMN_NAME') for dic in self.DBRunQuery(q)]
        for k in self.primary_key:
            if k not in columns:
                return True
        else:
            return False
            
    def invalid_keys_error(self,t):
        q = "SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='" + self.table_name + "'"
        columns = [dic.get('COLUMN_NAME') for dic in self.DBRunQuery(q)]
        for k in t:
            if k not in columns:
                return True
        else:
            return False
        
    def insert_error(self,t):
        for key in self.primary_key:
            if not t.__contains__(key):
                return "err1"
            table = self.find_by_primary_key([t.get(key)])
            if table != ():
                return "err2"
            
    def find_by_template(self,t,fields = None):
        if self.invalid_keys_error(t):
            return print("Error: find_by_template with invalid keys")
        w = self.templateToWhereClause(t)
        c = self.fieldsToColumName(fields)
        q = "SELECT " + c + " FROM " + self.table_name + " " + w + ";"
        result = self.DBRunQuery(q)
        return result
    
    def find_by_primary_key(self,string,fields = None):
        if self.primary_key_error():
            print("Error: primary keys are not a subset of the columns in the underlying file/table")
        dic = dict(zip(self.primary_key,string))
        w = self.templateToWhereClause(dic)
        c = self.fieldsToColumName(fields)
        q = "SELECT " + c + " FROM " + self.table_name + " " + w + ";"
        result = self.DBRunQuery(q)
        return result
        
    def insert(self,t):
        if self.invalid_keys_error(t):
            return print("Error: insert with invalid keys")
        if self.insert_error(t) == "err1":
            print("Error: insert without primary keys")
        elif self.insert_error(t) == "err2":
            print("Error: insert duplicate primary keys")
        s = self.templateToKeysValues(t)        
        q = "INSERT INTO " + self.table_name + s[0] + " VALUES " + s[1] + ";"        
        self.DBRunQuery(q)
        
    def delete(self,t):
        if self.invalid_keys_error(t):
            return print("delete with invalid keys")
        w = self.templateToWhereClause(t)
        q = "DELETE FROM " + self.table_name + " " + w + ";"
        self.DBRunQuery(q)
